In [8]:

####
# Importing Libarires


# Some links for documentation on packages
# https://python-docx.readthedocs.io/en/latest/user/documents.html
# https://docxtpl.readthedocs.io/en/latest/
# https://www.rikvoorhaar.com/python-docx/

# Installing docx in conda
# conda install docxtpl --channel conda-forge


from docx import Document, text
from docx.shared import Inches

import datetime


import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
import seaborn as sns

import stats_can


# Importing the PIL library
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
 

# Set Seaborn syle and colour pallete (to match AB gov standards)
sns.set(font="Arial")
sns.set_theme(style="white",
             palette = sns.color_palette([(0,170/255,210/255)],1),
             color_codes = True
             )
plt.rcParams['xtick.bottom'] = True
plt.rcParams['xtick.major.size'] = 2
plt.rcParams['xtick.major.width'] = 0.75


# In[2]:


# List of pargrapghs with errors
para_errors = []


# In[3]:


# num_to_to_string takes a float or int, rounds it, multiplies by 1000, and then adds "," every 3 digits
def num_to_string(x):
    if x<0:
        x = str(int(round(x,1)*1000))[1:]
        if len(x) > 3:
            x = x[0:(len(x)-3)]+','+x[-3:]
        if len(x) > 7:
            x = x[0:(len(x)-7)]+','+x[-7:]
        x = '-'+x
        return x
    else:
        x = str(int(round(x,1)*1000))
        if len(x) > 3:
            x = x[0:(len(x)-3)]+','+x[-3:]
        if len(x) > 7:
            x = x[0:(len(x)-7)]+','+x[-7:]
        return x
    
def float_to_string(x):
    x = str(x)
    x=x[0]+','+x[1:]
    return x
    
def change_width(ax, new_value) :
    for patch in ax.patches :
        current_width = patch.get_width()
        diff = current_width - new_value

        # we change the bar width
        patch.set_width(new_value)

        # we recenter the bar
        patch.set_x(patch.get_x() + diff * .5)
        
        
def change_height(ax, new_value):
    for patch in ax.patches :
        current_height = patch.get_height()
        diff = current_height - new_value

        # we change the bar width
        patch.set_height(new_value)

        # we recenter the bar
        patch.set_y(patch.get_y() + diff * .5)

def remove_line_break(text):
    list_of_words = text.split("\n")
    if len(list_of_words) == 1:
        return text.lower()
    else:
        returned_words = ""
        for words in list_of_words:
            returned_words = returned_words + words
        return returned_words.lower()
    
def add_pos_neg(number):
    if number > 0:
        return "+"+str(number)
    else:
        return str(number)

def chart_text(x):
    x = num_to_string(x)
    if x[0] == "-":
        x = x[1:]
    if len(x) == 1:
        return 0.25    
    if len(x) == 2:
        return 0.375
    if len(x) == 3:
        return 0.5
    if len(x) == 5:
        return 0.625
    if len(x) == 6:
        return 0.75
    if len(x) == 7:
        return 0.825
    if len(x) == 9:
        return 1
    
def file_date_text(x):
    if len(str(x.month)) == 1:
        month = "0"+str(x.month)
    else:
        month = str(x.month)
    return str(x.year)+"-"+month


# In[4]:


####
# Dates
# Calculate text for last month and the month before, and same month one year prior

# Date time object of last month
m1 = datetime.date.today()
m1 = m1.replace(
                year=m1.year if m1.month > 1 else m1.year - 1,
                month=m1.month - 1 if m1.month > 1 else 12,
                day=1
)

# Date time object of month before
m2 = datetime.date.today()
m2 = m2.replace(
                year=m2.year if m2.month > 2 else m2.year - 1,
                month= m2.month - 2 if m2.month > 2 else 12,
                day=1
)

# Date time object of same month one year prior
m3= datetime.date.today()
m3 = m3.replace(
                year=m3.year-1 if m3.month > 1 else m3.year - 2,
                month=m3.month - 1 if m3.month > 1 else 12,
                day=1
)

# Convert date time objects into a string for text
cur_m_text = m1.strftime('%B %Y')
pre_m_text = m2.strftime('%B %Y')
pre_y_text = m3.strftime('%B %Y')


# In[5]:


#### Data frames

# Download data frame for paragraphs 1 and 2 as well as charts 1 and 2

# data is from Table 14-10-0287-01      https://doi.org/10.25318/1410028701-eng
df_p1 = stats_can.sc.vectors_to_df(["v2062815",
                                    "v2063004",
                                    "v2063193",
                                    "v2063382",
                                    "v2063571",
                                    "v2063760",
                                    "v2063949",
                                    "v2064138",
                                    "v2064327",
                                    "v2064516",
                                    "v2064705"],
                                   periods = 13)


p1_dict = {"v2062815":"Canada",
           "v2063004":"Newfoundland and Labrador",
           "v2063193":"Prince Edward Island",
           "v2063382":"Nova Scotia",
           "v2063571":"New Brunswick",
           "v2063760":"Quebec",
           "v2063949":"Ontario",
           "v2064138":"Manitoba",
           "v2064327":"Saskatchewan",
           "v2064516":"Alberta",
           "v2064705":"British Columbia"
          }

df_p1.rename(columns = p1_dict, inplace = True)


# In[6]:


# Download data frame for paragraph 3

# data is from Table 14-10-0287-01      https://doi.org/10.25318/1410028701-eng

df_p3 = stats_can.sc.vectors_to_df(["v2064511",
                                    "v2064538",
                                    "v2064646",
                                    "v2064673"
                                    ],
                                   periods = 2)

p3_dict = {"v2064511":"Labour Force",
           "v2064538":"Labour Force - 15 to 24 year olds",
           "v2064646":"Labour Force - 25 to 54 year olds",
           "v2064673":"Labour Force - 55+ years"
}

df_p3.rename(columns = p3_dict, inplace = True)



# In[7]:


# Download data frame for paragraph 4 and chart 3

# data is from Table 14-10-0287-01      https://doi.org/10.25318/1410028701-eng

df_p4 = stats_can.sc.vectors_to_df(["v2064512",
                                    "v2064548",
                                    "v2064656",
                                    "v2064683",
                                    "v2064557",
                                    "v2064665",
                                    "v2064692"
                                    ],
                                   periods = 2)

p4_dict = {"v2064512":"Employment",
           "v2064548":"Employment - Male 15 to 24 year olds",
           "v2064656":"Employment - Male 25 to 54 year olds",
           "v2064683":"Employment - Male 55+ years",
           "v2064557":"Employment - Female 15 to 24 year olds",
           "v2064665":"Employment - Female 25 to 54 year olds",
           "v2064692":"Employment - Female 55+ years"
          }

df_p4.rename(columns = p4_dict, inplace = True)


# In[8]:


# Download data frame for paragraph 5
# data is from Table 14-10-0288-01      https://doi.org/10.25318/1410028801-eng

df_p5 = stats_can.sc.vectors_to_df(["v2067014","v2067015","v2067016"], periods = 13)

p5_dict = {"v2067014":"Public sector employees",
           "v2067015":"Private sector employees",
           "v2067016":"Self-employed"    
          }

df_p5.rename(columns = p5_dict, inplace = True)


# In[9]:


# Download data frame for paragraph 6

# data is from Table 14-10-0380-01      https://doi.org/10.25318/1410038001-eng

df_p6 = stats_can.sc.vectors_to_df(["v1235046238", "v1235046270", "v1235046302"], periods = 2)

p6_dict = {"v1235046238":"Alberta", 
           "v1235046270":"Calgary", 
           "v1235046302":"Edmonton"
          }

df_p6.rename(columns = p6_dict, inplace = True)


# In[10]:


# Download data frame for paragraph 7 and chart 4
# data is from Table 14-10-0355-01      https://doi.org/10.25318/1410035501-eng

df_p7 = stats_can.sc.vectors_to_df(["v2057776",
                                    "v2057777",
                                    "v2057778",
                                    "v2057779",
                                    "v2057780",
                                    "v2057782",
                                    "v2057783",
                                    "v2057784",
                                    "v2057785",
                                    "v2057786",
                                    "v2057787",
                                    "v2057788",
                                    "v2057789",
                                    "v2057790",
                                    "v2057791",
                                    "v2057792"],
                                   periods = 2)


p7_dict = {"v2057776":"Agriculture",
           "v2057777":"Forestry, fishing, mining, quarrying, oil and gas",
           "v2057778":"Utilities",
           "v2057779":"Construction",
           "v2057780":"Manufacturing",
           "v2057782":"Wholesale and retail trade",
           "v2057783":"Transportation and warehousing",
           "v2057784":"Finance, insurance, real estate, rental and leasing",
           "v2057785":"Professional, scientific and technical services",
           "v2057786":"Business, building and other support services",
           "v2057787":"Educational services",
           "v2057788":"Health care and social assistance",
           "v2057789":"Information, culture and recreation",
           "v2057790":"Accommodation and food services",
           "v2057791":"Other services (except public administration)",
           "v2057792":"Public administration"
          }

df_p7.rename(columns = p7_dict, inplace = True)


# In[11]:


# Download data frame for paragraph 8 and chart 5

# data is from Table 14-10-0421-01      https://doi.org/10.25318/1410042101-eng


df_p8 = stats_can.sc.vectors_to_df(["v1490040881",
                                    "v1490040896",
                                    "v1490040917",
                                    "v1490040935",
                                    "v1490040956",
                                    "v1490040989",
                                    "v1490041004",
                                    "v1490041019",
                                    "v1490041034",
                                    "v1490041043"
                                    ],
                                   periods = 13)

p8_dict = {"v1490040881":"Management occupations",
           "v1490040896":"Business, finance and administration\n occupations",
           "v1490040917":"Natural and applied sciences and related\n occupations",
           "v1490040935":"Health occupations",
           "v1490040956":"Occupations in education, law and social,\n community and government services",
           "v1490040989":"Occupations in art, culture, recreation\n and sport",
           "v1490041004":"Sales and service occupations",
           "v1490041019":"Trades, transport and equipment operators\n and related occupations",
           "v1490041034":"Natural resources, agriculture and related\n production occupations",
           "v1490041043":"Occupations in manufacturing and utilities"
            }

df_p8.rename(columns = p8_dict, inplace = True)


# In[12]:


# Download data frame for paragraph 9

# data is from Table 14-10-0387-01      https://doi.org/10.25318/1410038701-eng

df_p9 = stats_can.sc.vectors_to_df(["v1235065106",
                                    "v1235065134",
                                    "v1235065162",
                                    "v1235065190",
                                    "v1235065218",
                                    "v1235065246",
                                    "v1235065274"
                                    ],
                                   periods = 1)
p9_dict = {"v1235065106":"Lethbridge-Medicine Hat",
           "v1235065134":"Camrose-Drumheller",
           "v1235065162":"Calgary",
           "v1235065190":"Banff-Jasper-Rocky Mountain House and Athabasca-Grande Prairie-Peace River",
           "v1235065218":"Red Deer",
           "v1235065246":"Edmonton",
           "v1235065274":"Wood Buffalo-Cold Lake"
          }

df_p9.rename(columns = p9_dict, inplace = True)


# In[13]:


# Download data frame for table 

# data is from Table 14-10-0287-01      https://doi.org/10.25318/1410028701-eng
# data is from Table 14-10-0380-01      https://doi.org/10.25318/1410038001-eng
# data is from Table 14-10-0063-01      https://doi.org/10.25318/1410006301-eng


df_table = stats_can.sc.vectors_to_df(["v2064512",
                                       "v2064513",
                                       "v2064514",
                                       "v2064516",
                                       "v1235046280",
                                       "v1235046312",
                                       "v2064511",
                                       "v2064515",
                                       "v2064517",
                                       "v2163360"
                                    ],
                                   periods = 13)

table_dict = {"v2064512":"Employment",
              "v2064513":"Full Time Employment",
              "v2064514":"Part Time Employment",
              "v2064516":"Unemployment Rate (UR)",
              "v1235046280":"Calgary UR Three Month Moving Average",
              "v1235046312":"Edmonton UR Three Month Moving Average",
              "v2064511":"Labour Force",
              "v2064515":"Unemployment",
              "v2064517":"Participation Rate",
              "v2163360":"Average Weekly Wage",
             }

df_table.rename(columns = table_dict, inplace = True)


# In[14]:


# Download data frame for map

# data is from Table 14-10-0387-01     https://doi.org/10.25318/1410038701-eng

df_map = stats_can.sc.vectors_to_df(["v1235065106",
                                    "v1235065134",
                                    "v1235065162",
                                    "v1235065190",
                                    "v1235065218",
                                    "v1235065246",
                                    "v1235065274",                                                                    
                                    "v1235065109",
                                    "v1235065137",
                                    "v1235065165",
                                    "v1235065193",
                                    "v1235065221",
                                    "v1235065249",
                                    "v1235065277",
                                    "v1235065112",
                                    "v1235065140",
                                    "v1235065168",
                                    "v1235065196",
                                    "v1235065224",
                                    "v1235065252",
                                    "v1235065280"
                                     
                                     
                                    ],
                                   periods = 1)
map_dict = {"v1235065106":"LM UR",
           "v1235065134":"CD UR",
           "v1235065162":"CA UR",
           "v1235065190":"BJ UR",
           "v1235065218":"RD UR",
           "v1235065246":"ED UR",
           "v1235065274":"WC UR",           
           "v1235065109":"LM PR",
           "v1235065137":"CD PR",
           "v1235065165":"CA PR",
           "v1235065193":"BJ PR",
           "v1235065221":"RD PR",
           "v1235065249":"ED PR",
           "v1235065277":"WC PR",            
           "v1235065112":"LM ER",
           "v1235065140":"CD ER",
           "v1235065168":"CA ER",
           "v1235065196":"BJ ER",
           "v1235065224":"RD ER",
           "v1235065252":"ED ER",
           "v1235065280":"WC ER"
          }

df_map.rename(columns = map_dict, inplace = True)





# In[15]:


document = Document('Alberta Highlights Template - adjust.docx')


# In[16]:


# Change title of document for current date
document.tables[0].cell(1,1).paragraphs[1].text = cur_m_text


# In[17]:

last_dec=12-m1.month

try:
    
    list_levels = ["Employment","Full Time Employment","Part Time Employment","Labour Force","Unemployment"]

    levels = df_table[list_levels].iloc[12].apply(num_to_string)

    y_over_y_levels = df_table[list_levels].iloc[12]-df_table[list_levels].iloc[0]
    m_over_m_levels = df_table[list_levels].iloc[12]-df_table[list_levels].iloc[11]
    y_to_d_levels=df_table[list_levels].iloc[12]-df_table[list_levels].iloc[last_dec]

    y_over_y_levels = y_over_y_levels.apply(lambda x: "+" + num_to_string(x) if x>0 else num_to_string(x))
    m_over_m_levels = m_over_m_levels.apply(lambda x: "+" + num_to_string(x) if x>0 else num_to_string(x))
    y_to_d_levels = y_to_d_levels.apply(lambda x: "+" + num_to_string(x) if x>0 else num_to_string(x))


    list_rates = ["Unemployment Rate (UR)", "Edmonton UR Three Month Moving Average","Calgary UR Three Month Moving Average", "Participation Rate",]

    rates = df_table[list_rates].iloc[12].apply(round, ndigits = 1).apply(str).apply(lambda x: x +"%")

    y_over_y_rates = df_table[list_rates].iloc[12]-df_table[list_rates].iloc[0]
    m_over_m_rates = df_table[list_rates].iloc[12]-df_table[list_rates].iloc[11]
    y_to_d_rates=df_table[list_rates].iloc[12]-df_table[list_rates].iloc[last_dec]

    y_over_y_rates = y_over_y_rates.apply(lambda x: "+" + str(round(x,1)) +"*" if x>0 else str(round(x,1))+"*")
    m_over_m_rates = m_over_m_rates.apply(lambda x: "+" + str(round(x,1)) +"*" if x>0 else str(round(x,1))+"*")
    y_to_d_rates = y_to_d_rates.apply(lambda x: "+" + str(round(x,1)) +"*" if x>0 else str(round(x,1))+"*")

    wage = "$" + float_to_string(df_table["Average Weekly Wage"].iloc[12])

    y_over_y_wage = add_pos_neg(round(100*(df_table["Average Weekly Wage"].iloc[12]-df_table["Average Weekly Wage"].iloc[0])/df_table["Average Weekly Wage"].iloc[0],1))+"%"
    m_over_m_wage = add_pos_neg(round(100*(df_table["Average Weekly Wage"].iloc[12]-df_table["Average Weekly Wage"].iloc[11])/df_table["Average Weekly Wage"].iloc[11],1))+"%"
    y_to_d_wage = add_pos_neg(round(100*(df_table["Average Weekly Wage"].iloc[12]-df_table["Average Weekly Wage"].iloc[last_dec])/df_table["Average Weekly Wage"].iloc[last_dec],1))+"%"




    table_current = pd.concat([levels,rates])
    table_m_over_m = pd.concat([m_over_m_levels,m_over_m_rates])
    table_y_over_y = pd.concat([y_over_y_levels,y_over_y_rates])
    table_y_to_d= pd.concat([y_to_d_levels,y_to_d_rates])



    table_order = ["Employment",
                  "Full Time Employment",
                  "Part Time Employment",
                  "Unemployment Rate (UR)",
                   "Edmonton UR Three Month Moving Average",
                  "Calgary UR Three Month Moving Average",
                  "Labour Force",
                  "Unemployment",
                  "Participation Rate",
                  "Average Weekly Wage"]

    table_organized = pd.concat([table_current, table_m_over_m, table_y_over_y,  table_y_to_d],axis = 1)
    table_organized.columns = ["Current","Month-over-Month","Year-over-Year","Year-to-Date"]
    table_organized = table_organized.reindex(table_order)

    table_organized.loc["Average Weekly Wage"]["Current"] = wage
    table_organized.loc["Average Weekly Wage"]["Month-over-Month"] = m_over_m_wage
    table_organized.loc["Average Weekly Wage"]["Year-over-Year"] = y_over_y_wage
    table_organized.loc["Average Weekly Wage"]["Year-to-Date"] = y_to_d_wage


    # Change Table Date to current date
    document.tables[0].cell(3,2).paragraphs[0].text = cur_m_text

    # Add organized table data to table in word
    for row in range(len(table_order)):
        document.tables[0].cell(4+row,2).paragraphs[0].text = table_organized["Current"].iloc[row]
        document.tables[0].cell(4+row,3).paragraphs[0].text = table_organized["Month-over-Month"].iloc[row]
        document.tables[0].cell(4+row,4).paragraphs[0].text = table_organized["Year-over-Year"].iloc[row]
        document.tables[0].cell(4+row,5).paragraphs[0].text = table_organized["Year-to-Date"].iloc[row]
except:
    para_errors.append("Table")


# In[18]:


try:
    
    #Paragraph 1 sentence 1

    # Find unemployment rates for the different months
    cur_ur = df_p1["Alberta"].iloc[12]
    lastm_ur = df_p1["Alberta"].iloc[11]
    lasty_ur = df_p1["Alberta"].iloc[0]

    para1_sent1_part1 = "Alberta’s seasonally adjusted unemployment rate was "+str(cur_ur)+" per cent in "+cur_m_text +", "

    # Calculate change in the unemployment rate from last month
    ur_m_diff = round(cur_ur - lastm_ur,1)

    # Change text depending on if the unemployment increased/decreased
    if abs(ur_m_diff) == 1.0:
        if ur_m_diff >0:
            para1_sent1_part2 = "up "+ str(ur_m_diff) +" percentage point from the previous month "
        elif ur_m_diff <0:
            para1_sent1_part2 = "down "+ str(ur_m_diff)[1:] +" percentage point from the previous month "
    else:
        if ur_m_diff >0:
            para1_sent1_part2 = "up "+ str(ur_m_diff) +" percentage points from the previous month "
        elif ur_m_diff <0:
            para1_sent1_part2 = "down "+ str(ur_m_diff)[1:] +" percentage points from the previous month "
        else:
            para1_sent1_part2 = "unchanged from the previous month "

    # Calculate change in the unemployment rate from last year
    ur_y_diff = round(cur_ur - lasty_ur,1)

    # Change text depending on if the unemployment increased/decreased
    if abs(ur_y_diff) == 1.0:
        if ur_y_diff >0:
            para1_sent1_part3 = "and up "+ str(ur_y_diff) +" percentage point from the same month last year (Chart 1). "
        elif ur_y_diff <0:
            para1_sent1_part3 = "and down "+ str(ur_y_diff)[1:] +" percentage point from the same month last year (Chart 1). "
    else:
        if ur_y_diff >0:
            para1_sent1_part3 = "and up "+ str(ur_y_diff) +" percentage points from the same month last year (Chart 1). "
        elif ur_y_diff <0:
            para1_sent1_part3 = "and down "+ str(ur_y_diff)[1:] +" percentage points from the same month last year (Chart 1). "
        else:
            para1_sent1_part3 = "and unchanged from the same month last year (Chart 1). "

    # Form sentence
    para1 = para1_sent1_part1 + para1_sent1_part2 + para1_sent1_part3 

    document.paragraphs[2].text = para1

except:
    para_errors.append("paragraph 1")


# In[19]:


# Chart 1
try:

    # Add date for x-axis of charts
    df_p1["Date"] = list(df_p1.index)

    # Initialize the matplotlib figure
    # Size of plot
    f, ax = plt.subplots(figsize=(3.5, 0.91))
    # Chart title
    # Chart is automatically in middle
    ax.set_title('Chart 1 Alberta Unemployment Rate', size = 9,x = 0.25, color = 'black')
    # Set size of y and x axis labels
    ax.tick_params(axis='y', which='major', labelsize=5, color = 'black')
    ax.tick_params(axis='x', which='major', labelsize=5, color = 'black')
    # Create chart
    chart = sns.lineplot(x = "Date", y = "Alberta", data=df_p1)
    # Axis titles
    ax.set( ylabel="",
           xlabel="")
    # Format y axis with %
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals = 1))
    # Interval betweeen monts
    locator = mdates.MonthLocator(interval=2)
    chart.xaxis.set_major_locator(locator)
    # Remove graph borders
    sns.despine(left=True)
    # Set y axis range - set from 0 to 2 percentage points above maximum unemployment rate over period
    plt.ylim(df_p1["Alberta"].min()-1,df_p1["Alberta"].max()+1)
    # Save plot with high resolution
    plt.savefig("AB UR line.png",bbox_inches='tight', dpi = 1200)
    plt.close()



    # Add saved plot to document
    p = document.paragraphs[3]
    r = p.add_run()
    r.add_picture('AB UR line.png')
except:
    para_errors.append("chart 1")


# In[20]:


# Paragraph 2
try:
    # Paragraph 2 sentence 1

    # Grab last month and transpose data of unemployment rates, drop Canada, then sort people 
    df_p1_transposed = pd.DataFrame()
    df_p1_transposed["Data"] = df_p1.tail(1).transpose().drop(["Canada","Date"])
    df_p1_transposed["Prov"] = list(df_p1_transposed.index)

    # Sort by data then province so that Alberta is last if a tie
    df_p1_sorted = df_p1_transposed.sort_values(["Data","Prov"],ascending=[True, False])

    # Grab ordered list of provinces
    prov_ord = list(df_p1_sorted.index)

    # Find Alberta in list
    for i in range(10):
        if prov_ord[i] == "Alberta":
            rank = i+1

    number_of_ties = df_p1_sorted[df_p1_sorted["Data"]==df_p1_sorted["Data"].loc["Alberta"]]["Data"].count() - 1

    # Calculate if there are ties and keep track of them
    if number_of_ties > 0:
        tied = True

        # This means only one other province is tied with Alberta - just display their name
        if number_of_ties == 1:
             ties = " "+prov_ord[rank -2]
        else:
            ties = ""
            # for an index of tie in the range of the number of provinces tied with Alberta
            for tie in range(number_of_ties):
                # if the index equals the the number of ties -1 for range function
                if tie == (number_of_ties-1):
                    # - 2 is for skipping over ALberta (-1) and because range starts at 0 and ends 1 below (-1)
                    ties = ties+" and "+ prov_ord[rank -2- tie]
                else:
                    # If there are only 2 ties, we don't want the comma after
                    if number_of_ties == 2:
                        ties = ties+ " "+prov_ord[rank -2- tie]
                    # If there are more than 2 ties, include the comma
                    else:
                        ties = ties+ " "+prov_ord[rank -2- tie]+","

    else:
        tied = False

    # Dictionary of Alberta rank to text rank 
    rank_to_text = {1:"lowest", 2:"second lowest", 3:"third lowest", 4:"fourth lowest", 5:"fifth lowest",
     6:"fifth highest", 7:"fourth highest", 8:"third highest", 9:"second highest", 10:"highest"}

    # If Alberta's rank is the lowest
    if rank ==1:
        # Insert text rank
        para2_sent1 = "Alberta's unemployment rate was the "+ rank_to_text[rank]+" in Canada. "

    # If Alberta's rank is the second lowest
    elif rank ==2:
        # Insert text rank

        # This means tied for first lowest - Alberta will always be ranked lower if tied
        if tied:
            para2_sent1 = "Alberta's rate unemployment was tied for the lowest in Canada with"+ties+". "

        else: 
            para2_sent1 = "Alberta's rate unemployment was the second lowest in Canada after "+str(prov_ord[0])+". "
    # If Alberta's rank is not the lowest or second
    else:

        if tied:
            if rank != 10:
                # If not the highest, just say was tied for <>
                 # Insert text rank
                para2_sent1 = "Alberta's unemployment rate was tied for the "+ rank_to_text[rank]+" in Canada with" + ties+". "


            else:
                # If highest, say that it was tied for higher
                para2_sent1 = "Alberta's unemployment rate was tied for the highest in Canada with" + ties+". "



        else:
             # Insert text rank
            para2_sent1 = "Alberta's unemployment rate was the "+ rank_to_text[rank]+" in Canada. "




    # Paragraph 2 sentence 2
    # Canada unemployment rates
    cur_can_ur = df_p1["Canada"].iloc[12]
    lastm_can_ur = df_p1["Canada"].iloc[11]

    para2_sent2_part1 = "The national unemployment rate at "+ str(cur_can_ur)+" per cent was "

    # Calculate change
    ur_can_m_diff = round(cur_can_ur- lastm_can_ur,1)

    # Change sentence depending on if the change was an increase or a decrease
    if abs(ur_can_m_diff) == 1.0:
        if ur_can_m_diff > 0:
            para2_sent2_part2 = str(ur_can_m_diff)+ " percentage point higher than the previous month (Chart 2)." 
        elif ur_can_m_diff < 0:
            para2_sent2_part2 = str(ur_can_m_diff)[1:]+ " percentage point lower than the previous month (Chart 2)."

    else:
        if ur_can_m_diff > 0:
            para2_sent2_part2 = str(ur_can_m_diff)+ " percentage points higher than the previous month (Chart 2)." 
        elif ur_can_m_diff < 0:
            para2_sent2_part2 = str(ur_can_m_diff)[1:]+ " percentage points lower than the previous month (Chart 2)."
        else:
            para2_sent2_part2 = "unchanged compared to the previous month (Chart 2)."

    # Form sentence
    para2_sent2 = para2_sent2_part1 + para2_sent2_part2


    # Paragraph 1
    # Form paragraph and change word doc
    document.paragraphs[5].text = para2_sent1 + para2_sent2

except:
    para_errors.append("paragraph 2")


# In[21]:


# Chart 2 
try:

    # Arrange Data
    provinces = df_p1.drop("Date", axis = 1).tail(1).transpose()
    provinces.columns = ["Unemployment Rate"]
    provinces["Name"] = provinces.index
    provinces = provinces.sort_values("Unemployment Rate", ascending = False)

    # Provide colours
    colours = [(0,80/255,114/255) if (prov == "Alberta") else (212/255,0,114/255) if (prov == "Canada") else (95/255,206/255,234/255)  for prov in provinces.index]


    # Initialize the matplotlib figure
    # Size of chart
    #f, ax = plt.subplots(figsize=(2.4, 1.9))
    f, ax = plt.subplots(figsize=(2.2, 1.8))
    # Title
    #Setx= 0.2 for middle
    ax.set_title('Chart 2 Unemploment Rates by Province', size = 9, x = -0.2, color = 'black')
    # Make Chart
    sns.barplot(x="Unemployment Rate", y="Name", data=provinces, palette = colours, edgecolor = (0,0,0))


    # No axis titles
    ax.set( ylabel="",
           xlabel="")
    # Change text size of labels
    ax.tick_params(axis='both', which='major', labelsize=7, color = 'black')
    # Remove chart borders
    sns.despine(left=True, bottom=True)

    ax.axvline(0, color = (209/255, 212/255,211/255), linewidth=1)

    change_height(ax, 0.5)
    plt.setp(ax.patches, linewidth=0.5)

    ###
    # Add labels
    for p in ax.patches:
        ax.text(p.get_width()+ 1,
                p.get_y()+ p.get_height()*1.3, str(p.get_width())+"%", 
                fontsize=7, ha='center', va='bottom')


    #Set x width
    ax.set_xlim([0, provinces["Unemployment Rate"].max()+2])

    # Remove ticks and labels
    ax.set(xticklabels=[])
    ax.tick_params(bottom=False)

    ####



    # Save at a high resolution
    plt.savefig("Unemployment Rates.png",bbox_inches='tight', dpi = 1200)
    plt.close()

    p = document.paragraphs[6]
    r = p.add_run()
    r.add_picture("Unemployment Rates.png")

except:
    para_errors.append("chart 2")


# In[22]:


# Paragraph 3

try:

    labour_force = df_p3.transpose()

    # Rename dates to keep consistent month to month
    labour_force.columns = ["Last","Current"]
    # labels for display
    labour_force["Names"] = ["labour force", "15 to 24 year olds", "25 to 54 year olds","people aged 55 years and over"]
    labour_force["Change"] = labour_force["Current"] - labour_force["Last"]
    # numbers for display
    labour_force["Text Change"] = labour_force["Change"].apply(num_to_string)


    # Start text off by decribing how much the labour force increased to
    if labour_force["Change"].loc["Labour Force"] < 0:
        para3_sent1 = "In "+ cur_m_text+", the labour force decreased by " + labour_force["Text Change"].loc["Labour Force"][1:]+" to "+num_to_string(labour_force["Current"].loc["Labour Force"])+"."
    elif labour_force["Change"].loc["Labour Force"] > 0:
        para3_sent1 = "In "+ cur_m_text+", the labour force increased by " + labour_force["Text Change"].loc["Labour Force"]+" to "+num_to_string(labour_force["Current"].loc["Labour Force"])+"."
    else:
        para3_sent1 = "In "+ cur_m_text+", the total number of people in the labour force was unchanged at "+num_to_string(labour_force["Current"].loc["Labour Force"])+"."

    para3_sent2_part0 = " The number of labour force participants "


    # Make a list of increases/decreases/unchanged to labour force
    labour_force_increased = list(labour_force[(labour_force["Change"]>0) & (labour_force["Names"] != "labour force")]["Names"])
    labour_force_decreased = list(labour_force[(labour_force["Change"]<0) & (labour_force["Names"] != "labour force")]["Names"])
    labour_force_unchanged = list(labour_force[(labour_force["Change"]==0) & (labour_force["Names"] != "labour force")]["Names"])

    # Count number of increases and add the demographic and number to text
    if len(labour_force_increased) == 1:
        para3_sent2_part1 = "increased among "
        para3_sent2_part1 = para3_sent2_part1+ labour_force_increased[0]+" (+"+ labour_force[labour_force["Names"] == labour_force_increased[0]]["Text Change"].iloc[0] +")"

    elif len(labour_force_increased) == 2:
        para3_sent2_part1 = "increased among "
        para3_sent2_part1 = para3_sent2_part1+ labour_force_increased[0] +" (+"+ labour_force[labour_force["Names"] == labour_force_increased[0]]["Text Change"].iloc[0] +") and "
        para3_sent2_part1 = para3_sent2_part1+ labour_force_increased[1] +" (+"+ labour_force[labour_force["Names"] == labour_force_increased[1]]["Text Change"].iloc[0] +")"

    elif len(labour_force_increased) == 3:
        para3_sent2_part1 = "increased among "
        para3_sent2_part1 = para3_sent2_part1+ labour_force_increased[0] +" (+"+ labour_force[labour_force["Names"] == labour_force_increased[0]]["Text Change"].iloc[0] +"), "
        para3_sent2_part1 = para3_sent2_part1+ labour_force_increased[1] +" (+"+ labour_force[labour_force["Names"] == labour_force_increased[1]]["Text Change"].iloc[0] +"), and "
        para3_sent2_part1 = para3_sent2_part1+ labour_force_increased[2] +" (+"+ labour_force[labour_force["Names"] == labour_force_increased[2]]["Text Change"].iloc[0] +")"
    else:
        para3_sent2_part1 = ""


    # Count number of decreases and add the demographic and number to text    
    if len(labour_force_decreased) == 1:
        para3_sent2_part2 = "decreased for "
        para3_sent2_part2 = para3_sent2_part2+ labour_force_decreased[0] +" ("+ labour_force[labour_force["Names"] == labour_force_decreased[0]]["Text Change"].iloc[0] +")"

    elif len(labour_force_decreased) == 2:
        para3_sent2_part2 = "decreased for "
        para3_sent2_part2 = para3_sent2_part2+ labour_force_decreased[0] +" ("+ labour_force[labour_force["Names"] == labour_force_decreased[0]]["Text Change"].iloc[0] +") and "
        para3_sent2_part2 = para3_sent2_part2+ labour_force_decreased[1] +" ("+ labour_force[labour_force["Names"] == labour_force_decreased[1]]["Text Change"].iloc[0] +")"

    elif len(labour_force_decreased) == 3:
        para3_sent2_part2 = "decreased for "
        para3_sent2_part2 = para3_sent2_part2+ labour_force_decreased[0] +" ("+ labour_force[labour_force["Names"] == labour_force_decreased[0]]["Text Change"].iloc[0] +"), "
        para3_sent2_part2 = para3_sent2_part2+ labour_force_decreased[1] +" ("+ labour_force[labour_force["Names"] == labour_force_decreased[1]]["Text Change"].iloc[0] +"), and "
        para3_sent2_part2 = para3_sent2_part2+ labour_force_decreased[2] +" ("+ labour_force[labour_force["Names"] == labour_force_decreased[2]]["Text Change"].iloc[0] +")"
    else:
        para3_sent2_part2 = ""

    # Count number of unchagned and add the demographic and number to text
    if len(labour_force_unchanged) == 1:
        para3_sent2_part3 = "was unchanged for "
        para3_sent2_part3 = para3_sent2_part3+ labour_force_unchanged[0]

    elif len(labour_force_unchanged) == 2:
        para3_sent2_part3 = "was unchanged for "
        para3_sent2_part3 = para3_sent2_part3+ labour_force_unchanged[0] +" and "
        para3_sent2_part3 = para3_sent2_part3+ labour_force_unchanged[1]

    elif len(labour_force_unchanged) == 3:
        para3_sent2_part3 = "was unchanged for "
        para3_sent2_part3 = para3_sent2_part3+ labour_force_unchanged[0] +", "
        para3_sent2_part3 = para3_sent2_part3+ labour_force_unchanged[1] +", and "
        para3_sent2_part3 = para3_sent2_part3+ labour_force_unchanged[2]
    else:
        para3_sent2_part3 = ""

    # If no increases in labour fource
    if para3_sent2_part1 == "":
        # If no decreases in labour force

        if para3_sent2_part2 == "":
            para3_sent2 = para3_sent2_part0 + para3_sent2_part3 + "."

        # If decreases in labour force/ no no change
        elif para3_sent2_part3 == "":
            para3_sent2 = para3_sent2_part0 + para3_sent2_part2 + "."

        # If decrease in labour force and a no change
        else: 
            para3_sent2 = para3_sent2_part0 + para3_sent2_part2 +" and "+ para3_sent2_part3 + "."

    # If increases in labour force        
    else:

        #If no decrease in labour force
        if para3_sent2_part2 == "":

            # If no no change
            if para3_sent2_part3 == "":
                para3_sent2 = para3_sent2_part0 + para3_sent2_part1 + "."

            # If no change
            else:
                para3_sent2 = para3_sent2_part0 + para3_sent2_part1 + " and " + para3_sent2_part3 + "."

        #If increase in labour force
        else:

            # If no no change
            if para3_sent2_part3 == "":
                para3_sent2 = para3_sent2_part0 + para3_sent2_part1 + " and " + para3_sent2_part2 + "."

            # If no change
            else:
                 para3_sent2 = para3_sent2_part0 + para3_sent2_part1 + ", " + para3_sent2_part2 + " and " + para3_sent2_part3 + "."


    para3 = para3_sent1 + para3_sent2     

    document.paragraphs[8].text = para3

except:
    para_errors.append("paragraph 3")


# In[23]:


# Paragraph 4

try:

    employment = df_p4.transpose()

    # Aggregate ages
    employment.loc["15 to 24 year olds"] = employment.loc["Employment - Male 15 to 24 year olds"] +employment.loc["Employment - Female 15 to 24 year olds"] 
    employment.loc["25 to 54 year olds"] = employment.loc["Employment - Female 25 to 54 year olds"] + employment.loc["Employment - Male 25 to 54 year olds"]
    employment.loc["55+ years"] = employment.loc["Employment - Female 55+ years"] + employment.loc["Employment - Male 55+ years"]

    # Drop the age and sex intersection
    employment.drop(["Employment - Male 15 to 24 year olds",
                     "Employment - Female 15 to 24 year olds",
                     "Employment - Female 25 to 54 year olds",
                     "Employment - Male 25 to 54 year olds",
                     "Employment - Female 55+ years",
                     "Employment - Male 55+ years"
                     ], inplace = True)

    # Rename dates to keep consistent month to month
    employment.columns = ["Last","Current"]
    # labels for display
    employment["Names"] = ["employment", "15 to 24 year olds", "25 to 54 year olds","people aged 55 years and over"]
    employment["Change"] = employment["Current"] - employment["Last"]
    # numbers for display
    employment["Text Change"] = employment["Change"].apply(num_to_string)


    # Start text off by saying how much employment increased/decreased by.
    if employment["Change"].loc["Employment"] < 0:
        para4_sent1 = "Month-over-month, employment decreased by " + employment["Text Change"].loc["Employment"][1:]+"."
    elif employment["Change"].loc["Employment"] > 0:
        para4_sent1 = "Month-over-month, employment increased by " + employment["Text Change"].loc["Employment"]+"."
    else:
        para4_sent1 = "Month-over-month, employment was unchanged."

    para4_sent2_part0 = " Employment "

    # Create a list of increases and decreases
    employment_increased = list(employment[(employment["Change"]>0) & (employment["Names"] != "employment")]["Names"])
    employment_decreased = list(employment[(employment["Change"]<0) & (employment["Names"] != "employment")]["Names"])
    employment_unchanged = list(employment[(employment["Change"]==0) & (employment["Names"] != "employment")]["Names"])

    # Count number of increases and add the demographic and number to text
    if len(employment_increased) == 1:
        para4_sent2_part1 = "grew for "
        para4_sent2_part1 = para4_sent2_part1+ employment_increased[0]

    elif len(employment_increased) == 2:
        para4_sent2_part1 = "grew for "
        para4_sent2_part1 = para4_sent2_part1+ employment_increased[0] +" and "
        para4_sent2_part1 = para4_sent2_part1+ employment_increased[1]

    elif len(employment_increased) == 3:
        para4_sent2_part1 = "grew for "
        para4_sent2_part1 = para4_sent2_part1+ employment_increased[0] +", "
        para4_sent2_part1 = para4_sent2_part1+ employment_increased[1] +", and "
        para4_sent2_part1 = para4_sent2_part1+ employment_increased[2]
    else:
        para4_sent2_part1 = ""


    # Count number of decreases and add the demographic and number to text    
    if len(employment_decreased) == 1:
        para4_sent2_part2 = "fell among "
        para4_sent2_part2 = para4_sent2_part2+ employment_decreased[0]

    elif len(employment_decreased) == 2:
        para4_sent2_part2 = "fell among "
        para4_sent2_part2 = para4_sent2_part2+ employment_decreased[0] +" and "
        para4_sent2_part2 = para4_sent2_part2+ employment_decreased[1] 

    elif len(employment_decreased) == 3:
        para4_sent2_part2 = "fell among "
        para4_sent2_part2 = para4_sent2_part2+ employment_decreased[0] +", "
        para4_sent2_part2 = para4_sent2_part2+ employment_decreased[1] +", and "
        para4_sent2_part2 = para4_sent2_part2+ employment_decreased[2] 
    else:
        para4_sent2_part2 = ""

    # Count number of no changes and add the demographic and number to text
    if len(employment_unchanged) == 1:
        para4_sent2_part3 = "was unchanged for "
        para4_sent2_part3 = para4_sent2_part3+ employment_unchanged[0]

    elif len(employment_unchanged) == 2:
        para4_sent2_part3 = "was unchanged for "
        para4_sent2_part3 = para4_sent2_part3+ employment_unchanged[0] +" and "
        para4_sent2_part3 = para4_sent2_part3+ employment_unchanged[1]

    elif len(employment_unchanged) == 3:
        para4_sent2_part3 = "was unchanged for "
        para4_sent2_part3 = para4_sent2_part3+ employment_unchanged[0] +", "
        para4_sent2_part3 = para4_sent2_part3+ employment_unchanged[1] +", and "
        para4_sent2_part3 = para4_sent2_part3+ employment_unchanged[2]
    else:
        para4_sent2_part3 = ""

    # If no increases in employment
    if para4_sent2_part1 == "":
        # If no decreases in employment

        if para4_sent2_part2 == "":
            para4_sent2 = para4_sent2_part0 + para4_sent2_part3 + "."

        # If decreases in employment/ no no change
        elif para4_sent2_part3 == "":
            para4_sent2 = para4_sent2_part0 + para4_sent2_part2 + "."

        # If decrease in employment and a no change
        else: 
            para4_sent2 = para4_sent2_part0 + para4_sent2_part2 +" and "+ para4_sent2_part3 + "."

    # If increases in employment        
    else:

        #If no decrease in employment
        if para4_sent2_part2 == "":

            # If no no change
            if para4_sent2_part3 == "":
                para4_sent2 = para4_sent2_part0 + para4_sent2_part1 + "."

            # If no change
            else:
                para4_sent2 = para4_sent2_part0 + para4_sent2_part1 + " and " + para4_sent2_part3 + "."

        #If increase in employment
        else:

            # If no no change
            if para4_sent2_part3 == "":
                para4_sent2 = para4_sent2_part0 + para4_sent2_part1 + " and " + para4_sent2_part2 + "."

            # If no change
            else:
                 para4_sent2 = para4_sent2_part0 + para4_sent2_part1 + ", " + para4_sent2_part2 + " and " + para4_sent2_part3 + "."


    para4 = para4_sent1 + para4_sent2    

    document.paragraphs[10].text = para4

except:
    para_errors.append("paragraph 4")


# In[24]:


# Chart 3

try:

    employment_ag = df_p4.transpose().drop("Employment")
    employment_ag.columns = ["Last", "Current"]
    employment_ag["Change"] = employment_ag["Current"] -  employment_ag["Last"]
    employment_ag["Age"] = ["15 to 24 years old","15 to 24 years old", "25 to 54 years old", "25 to 54 years old", "55+ years old", "55+ years old"]
    employment_ag["Sex"] = ["Men", "Women", "Men",  "Women","Men", "Women"]


    f, ax = plt.subplots(figsize=(3.8, 0.7))
    # Title
    #Set x = 0.5 for middle of chart
    ax.set_title('Chart 3 Monthly Change in Employment\nby Age and Gender', size = 9, loc = 'left', color = 'black')
    # Make Chart

    sns.barplot(data = employment_ag, x = "Age", y ="Change", hue = "Sex",
               palette = sns.color_palette([(0,80/255,114/255),(95/255,206/255,234/255)],2),
               edgecolor = (0,0,0))
    plt.legend(title='', loc = "lower center", bbox_to_anchor = (0.5,-0.7), fontsize = 7, frameon = False, ncol = 2)

    # No axis titles
    ax.set( ylabel="",
           xlabel="")
    # Change text size of labels
    ax.tick_params(axis='both', which='major', labelsize=7, color = 'black')
    # Remove chart borders
    sns.despine(left=True, bottom=True)

    ax.axhline(0, color = (209/255, 212/255,211/255), linewidth=1)

    change_width(ax, 0.35)
    plt.setp(ax.patches, linewidth=0.5)

    max_height = employment_ag["Change"].max()*1.4
    if employment_ag["Change"].min()< 0:
        min_height = employment_ag["Change"].min()*1.4
    else:
        min_height = employment_ag["Change"].min()*0

    for p in ax.patches:
        if p.get_height() < 0:
            ax.text(p.get_x() + p.get_width()/2,
                p.get_height() - (max_height - min_height)/6, num_to_string(p.get_height()), 
                fontsize=7, ha='center', va='bottom')
        else:
            ax.text(p.get_x() + p.get_width()/2,
                p.get_height(), num_to_string(p.get_height()), 
                fontsize=7, ha='center', va='bottom')

    # Remove ticks and labels
    ax.set(yticklabels=[])
    ax.tick_params(bottom=False)

    ax.set_ylim([min_height,max_height])

    plt.savefig("AB Employment by Age and Gender.png",bbox_inches='tight', dpi = 1200)
    plt.close()

    p = document.paragraphs[11]
    r = p.add_run()
    r.add_picture("AB Employment by Age and Gender.png")

except:
    para_errors.append("chart 3")


# In[25]:


# paragraph 5

try:

    para5_sent1 = "Month-over-month, employment"

    # Calculate changes
    df_p5.loc["M/M Change"] = round(df_p5.iloc[12] - df_p5.iloc[11],1)
    df_p5.loc["Y/Y Change"] = round(df_p5.iloc[12] - df_p5.iloc[0],1)

    # Assign year over year changes for storage for later
    pub_y = df_p5.loc["Y/Y Change","Public sector employees"]
    pri_y = df_p5.loc["Y/Y Change","Private sector employees"]

    # Make a new data frame with data organized in a more logical manner
    pub_pri_sel = pd.DataFrame( data = {"Public sector employees":
                                            [df_p5.loc["M/M Change","Public sector employees"],
                                             df_p5.loc["Y/Y Change","Public sector employees"],
                                            "in the public sector"],
                                       "Private sector employees":
                                            [df_p5.loc["M/M Change","Private sector employees"],
                                             df_p5.loc["Y/Y Change","Private sector employees"],
                                            "in the private sector"],
                                       "Self-employed":
                                            [df_p5.loc["M/M Change","Self-employed"],
                                            df_p5.loc["Y/Y Change","Self-employed"],
                                            "among the self-employed"]},
                              index = ["M/M", "Y/Y", "Names"]).transpose()

    # Change numbers to text for process
    pub_pri_sel["M/M Text Change"] = pub_pri_sel["M/M"].apply(num_to_string)
    pub_pri_sel["Y/Y Text Change"] = pub_pri_sel["Y/Y"].apply(num_to_string)

    # Start of text
    para5_sent1_part0 = "Month-over-month, employment "

    # Make a list of all the increases/decreases/unchanged
    pub_pri_sel_increased = list(pub_pri_sel[pub_pri_sel["M/M"]>0]["Names"])
    pub_pri_sel_decreased = list(pub_pri_sel[pub_pri_sel["M/M"]<0]["Names"])
    pub_pri_sel_unchanged = list(pub_pri_sel[pub_pri_sel["M/M"]==0]["Names"])

    # Count number of increases and add the sector and number to text
    if len(pub_pri_sel_increased) == 1:
        para5_sent1_part1 = "increased "
        para5_sent1_part1 = para5_sent1_part1+ pub_pri_sel_increased[0]+" (+"+ pub_pri_sel[pub_pri_sel["Names"] == pub_pri_sel_increased[0]]["M/M Text Change"].iloc[0] +")"

    elif len(pub_pri_sel_increased) == 2:
        para5_sent1_part1 = "increased "
        para5_sent1_part1 = para5_sent1_part1+ pub_pri_sel_increased[0] +" (+"+ pub_pri_sel[pub_pri_sel["Names"] == pub_pri_sel_increased[0]]["M/M Text Change"].iloc[0] +") and "
        para5_sent1_part1 = para5_sent1_part1+ pub_pri_sel_increased[1] +" (+"+ pub_pri_sel[pub_pri_sel["Names"] == pub_pri_sel_increased[1]]["M/M Text Change"].iloc[0] +")"

    elif len(pub_pri_sel_increased) == 3:
        para5_sent1_part1 = "increased "
        para5_sent1_part1 = para5_sent1_part1+ pub_pri_sel_increased[0] +" (+"+ pub_pri_sel[pub_pri_sel["Names"] == pub_pri_sel_increased[0]]["M/M Text Change"].iloc[0] +"), "
        para5_sent1_part1 = para5_sent1_part1+ pub_pri_sel_increased[1] +" (+"+ pub_pri_sel[pub_pri_sel["Names"] == pub_pri_sel_increased[1]]["M/M Text Change"].iloc[0] +"), and "
        para5_sent1_part1 = para5_sent1_part1+ pub_pri_sel_increased[2] +" (+"+ pub_pri_sel[pub_pri_sel["Names"] == pub_pri_sel_increased[2]]["M/M Text Change"].iloc[0] +")"
    else:
        para5_sent1_part1 = ""


    # Count number of decreases and add the sector and number to text    
    if len(pub_pri_sel_decreased) == 1:
        para5_sent1_part2 = "decreased "
        para5_sent1_part2 = para5_sent1_part2+ pub_pri_sel_decreased[0] +" ("+ pub_pri_sel[pub_pri_sel["Names"] == pub_pri_sel_decreased[0]]["M/M Text Change"].iloc[0] +")"

    elif len(pub_pri_sel_decreased) == 2:
        para5_sent1_part2 = "decreased "
        para5_sent1_part2 = para5_sent1_part2+ pub_pri_sel_decreased[0] +" ("+ pub_pri_sel[pub_pri_sel["Names"] == pub_pri_sel_decreased[0]]["M/M Text Change"].iloc[0] +") and "
        para5_sent1_part2 = para5_sent1_part2+ pub_pri_sel_decreased[1] +" ("+ pub_pri_sel[pub_pri_sel["Names"] == pub_pri_sel_decreased[1]]["M/M Text Change"].iloc[0] +")"

    elif len(pub_pri_sel_decreased) == 3:
        para5_sent1_part2 = "decreased "
        para5_sent1_part2 = para5_sent1_part2+ pub_pri_sel_decreased[0] +" ("+ pub_pri_sel[pub_pri_sel["Names"] == pub_pri_sel_decreased[0]]["M/M Text Change"].iloc[0] +"), "
        para5_sent1_part2 = para5_sent1_part2+ pub_pri_sel_decreased[1] +" ("+ pub_pri_sel[pub_pri_sel["Names"] == pub_pri_sel_decreased[1]]["M/M Text Change"].iloc[0] +"), and "
        para5_sent1_part2 = para5_sent1_part2+ pub_pri_sel_decreased[2] +" ("+ pub_pri_sel[pub_pri_sel["Names"] == pub_pri_sel_decreased[2]]["M/M Text Change"].iloc[0] +")"
    else:
        para5_sent1_part2 = ""

    # Count number of no changes and add the sector and number to text
    if len(pub_pri_sel_unchanged) == 1:
        para5_sent1_part3 = "was unchanged "
        para5_sent1_part3 = para5_sent1_part3+ pub_pri_sel_unchanged[0]

    elif len(pub_pri_sel_unchanged) == 2:
        para5_sent1_part3 = "was unchanged "
        para5_sent1_part3 = para5_sent1_part3+ pub_pri_sel_unchanged[0] +" and "
        para5_sent1_part3 = para5_sent1_part3+ pub_pri_sel_unchanged[1]

    elif len(pub_pri_sel_unchanged) == 3:
        para5_sent1_part3 = "was unchanged "
        para5_sent1_part3 = para5_sent1_part3+ pub_pri_sel_unchanged[0] +", "
        para5_sent1_part3 = para5_sent1_part3+ pub_pri_sel_unchanged[1] +", and "
        para5_sent1_part3 = para5_sent1_part3+ pub_pri_sel_unchanged[2]
    else:
        para5_sent1_part3 = ""

    # Join everything together for the first sentence
    # If no increases in labour fource
    if para5_sent1_part1 == "":
        # If no decreases in labour force

        if para5_sent1_part2 == "":
            para5_sent1 = para5_sent1_part0 + para5_sent1_part3 + ". "

        # If decreases in labour force/ no no change
        elif para5_sent1_part3 == "":
            para5_sent1 = para5_sent1_part0 + para5_sent1_part2 + ". "

        # If decrease in labour force and a no change
        else: 
            para5_sent1 = para5_sent1_part0 + para5_sent1_part2 +" and "+ para5_sent1_part3 + ". "

    # If increases in labour force        
    else:

        #If no decrease in labour force
        if para5_sent1_part2 == "":

            # If no no change
            if para5_sent1_part3 == "":
                para5_sent1 = para5_sent1_part0 + para5_sent1_part1 + ". "

            # If no change
            else:
                para5_sent1 = para5_sent1_part0 + para5_sent1_part1 + " and " + para5_sent1_part3 + ". "

        #If increase in labour force
        else:

            # If no no change
            if para5_sent1_part3 == "":
                para5_sent1 = para5_sent1_part0 + para5_sent1_part1 + " and " + para5_sent1_part2 + ". "

            # If no change
            else:
                 para5_sent1 = para5_sent1_part0 + para5_sent1_part1 + ", " + para5_sent1_part2 + " and " + para5_sent1_part3 + ". "

    # Calculate year over year changes                

    para5_sent2_part1 = "On a year-over-year basis, "

    if pub_y and pri_y == 0:
        para5_sent2_part2 = "private and public sector employment was unchanged." 

    else:
        if pri_y >0:
            para5_sent2_part2_1 = "private sector employment grew by "+ num_to_string(pri_y)+" and "
        elif pri_y <0:
            para5_sent2_part2_1 = "private sector employment fell by "+ num_to_string(pri_y)[1:]+" and "
        else:
            para5_sent2_part2_1 = "private sector employment was unchanged and "

        if pri_y >0:
            para5_sent2_part2_2 = "public sector employment increased by "+ num_to_string(pub_y)+"."
        elif pri_y <0:
            para5_sent2_part2_2 = "public sector employment decreased by "+ num_to_string(pub_y)[1:]+"."
        else:
            para5_sent2_part2_2 = "public sector employment was unchanged."

        para5_sent2_part2 = para5_sent2_part2_1+para5_sent2_part2_2

    # Join everything together and add paragraph to document
    para5_sent2 = para5_sent2_part1 + para5_sent2_part2

    para5 = para5_sent1 + para5_sent2

    document.paragraphs[14].text = para5

except:
    para_errors.append("paragraph 5")


# In[26]:


# Paragraph 6

try:

    # Calculate changes and the employment growth in Alberta outside of Calgary and Edmonton
    cma = df_p6.transpose()

    cma.columns = ["Last", "Current"]
    cma.loc["Rest of Alberta"] =  cma.loc["Alberta"] - cma.loc["Calgary"] - cma.loc["Edmonton"]
    cma["Change"] = cma["Current"] -  cma["Last"]

    # Round the data
    cma_ed = round(cma["Change"].loc["Edmonton"],1)
    cma_ca = round(cma["Change"].loc["Calgary"],1)
    cma_ra = round(cma["Change"].loc["Rest of Alberta"],1) 

    # Start text
    para6_sent1_part1 = "Compared to the previous month, the seasonally adjusted three-month moving average for employment "

    # Edmonton increase/decrease
    if cma_ed >0:
        para6_sent1_part2 = "grew by "+ num_to_string(cma_ed)+" in Edmonton and "
    elif cma_ed <0:
        para6_sent1_part2 = "fell by "+ num_to_string(cma_ed)[1:]+" in Edmonton and "
    else:
        para6_sent1_part2 = "was unchanged in Edmonton and "

    # Calgary increase/decrease
    if cma_ca >0:
        para6_sent1_part3 = "rose by "+ num_to_string(cma_ca)+" in Calgary. "
    elif cma_ca <0:
        para6_sent1_part3 = "shrunk by "+ num_to_string(cma_ca)[1:]+" in Calgary. "
    else:
        para6_sent1_part3 = "was unchanged in Calgary. "

    # Join first sentence    
    para6_sent1 = para6_sent1_part1 + para6_sent1_part2 + para6_sent1_part3    

    # Calculate rest of Alberta and form sentence
    if cma_ra >0:
        para6_sent2 = "It grew by "+ num_to_string(cma_ra)+" in the rest of Alberta."
    elif cma_ra <0:
        para6_sent2 = "It fell by "+ num_to_string(cma_ra)[1:]+" in the rest of Alberta."
    else:
        para6_sent2 = "It was unchanged in the rest of Alberta."

    # Join it all together and add the paragraph
    para6 =  para6_sent1 +  para6_sent2

    document.paragraphs[16].text = para6

except:
    para_errors.append("paragraph 6")


# In[27]:


# Paragraph 7

try:

    # Calculate changes and sort by it
    industry = df_p7.transpose()
    industry.columns = ["Last","Current"]
    industry["Names"] = industry.index
    industry["Change"] = industry["Current"] - industry["Last"]
    industry = industry.sort_values("Change", ascending = False)

    # Set counter to 0
    ind_empl_count = 0

    # Count the number of positive increases
    for ind_row in range(len(industry)):
        if round(industry["Change"].iloc[ind_row],1) > 0:
            ind_empl_count = ind_empl_count + 1

    ind_empl_count_dict = {1:"one", 2:"two", 3:"three", 4:"four", 5:"five",
                           6:"six", 7:"seven", 8:"eight", 9:"nine", 10:"ten",
                           11:"11", 12:"12", 13:"13", 14:"14", 15:"15", 16:"16"}        
            
            
    
    # Statement when no increases
    if ind_empl_count == 0:
         para7_sent1 = "Compared to the previous month, employment rose in none of the industries. "        

    # Statement when there is at least one increase
    else:
        para7_sent1 = "In "+cur_m_text+", employment rose in "+ind_empl_count_dict[ind_empl_count]+" out of 16 industries compared to the previous month. "        

    # Saying industries with increases    
    if ind_empl_count == 0:
        para7_sent2 = "The industry with the largest decrease in employment in "+cur_m_text+" was "+ industry["Names"].iloc[15].lower() +" (Chart 4)."
    elif ind_empl_count == 1:
        para7_sent2 = "The only industry to see employment gains was " + industry["Names"].iloc[0].lower()+" (Chart 4)."
    elif ind_empl_count == 2:
        para7_sent2 = "The industries that saw employment gains were: " + industry["Names"].iloc[0].lower()+"; and " + industry["Names"].iloc[1].lower()+ " (Chart 4)."
    elif ind_empl_count == 3:
        para7_sent2 = "The industries that saw employment gains were: " + industry["Names"].iloc[0].lower()+"; "+ industry["Names"].iloc[1].lower()+"; and " + industry["Names"].iloc[2].lower()+ " (Chart 4)."
    else:
        para7_sent2 = "The industries with the most employment gains were: " + industry["Names"].iloc[0].lower()+"; "+ industry["Names"].iloc[1].lower()+"; and " + industry["Names"].iloc[2].lower()+ " (Chart 4)."


    para7 = para7_sent1+ para7_sent2 

    document.paragraphs[17].text = para7

except:
    para_errors.append("paragraph 7")


# In[28]:


# Chart 4

try: 
    chart4_pos = 0.2

    # Initialize the matplotlib figure
    # Size of chart
    #f, ax = plt.subplots(figsize=(1.2, 3.9))
    f, ax = plt.subplots(figsize=(1.2, 3.4))
    # Title
    # Set x = 0 for middle, remove loc
    ax.set_title('Chart 4 Monthly Employment Growth\nBy Industry', size = 9, loc = 'left', x = -2.5, color = 'black')
    # Make Chart
    sns.barplot(x="Change", y="Names", data=industry,palette = sns.color_palette([(0,170/255,210/255)],1),
                edgecolor = (0,0,0))
    # No axis titles
    ax.set( ylabel="",
           xlabel="")
    # Change text size of labels
    ax.tick_params(axis='both', which='major', labelsize=7, color = 'black')
    # Remove chart borders
    sns.despine(left=True, bottom=True)




    if industry['Change'].max() >0:
        x_max = industry['Change'].max()+chart_text(industry['Change'].max())/chart4_pos+ 0.125/chart4_pos
    else:
        x_max = 0
    if industry['Change'].min() <0:
        x_min = industry['Change'].min()-chart_text(industry['Change'].min())/chart4_pos-0.25/chart4_pos
    else:
        x_min = 0
    ax.set_xlim(left = -abs(x_min)**1.06, right = x_max**1.06)

    chart4_pos2 = 5/(x_max-x_min)


    ###
    # Add labels
    for p in ax.patches:
        if p.get_width()<0:
            ax.text(p.get_width()-(chart_text(p.get_width())/chart4_pos2)- 0.25/chart4_pos2,
                    p.get_y()+ p.get_height()*0.8, num_to_string(p.get_width()), 
                fontsize=6, ha='center', va='bottom')
        else:
            ax.text(p.get_width()+(chart_text(p.get_width())/chart4_pos2)+ 0.125/chart4_pos2,
                    p.get_y()+ p.get_height()*0.8, num_to_string(p.get_width()), 
                fontsize=6, ha='center', va='bottom')

    plt.setp(ax.patches, linewidth=0.5)        
    ax.axvline(0, color = (209/255, 212/255,211/255), linewidth=1)        

    # Remove ticks and labels
    ax.set(xticklabels=[])
    ax.tick_params(bottom=False)

    ####



    # Save at a high resolution
    plt.savefig("AB Industries.png",bbox_inches='tight', dpi = 1200)
    plt.close()

    p = document.paragraphs[18]
    r = p.add_run()
    r.add_picture("AB Industries.png")

except:
    para_errors.append("chart 4")


# In[29]:


# Paragraph 8

try:
 
    # Grab only this year and last year
    df_p8 = df_p8.iloc[[0,12]]

    occupations = df_p8.transpose()
    occupations.columns = ["Last","Current"]
    occupations["Names"] = occupations.index
    occupations["Change"] = occupations["Current"] - occupations["Last"]
    occupations = occupations.sort_values("Change", ascending = False)
    occupations["Paragraph Names"] = occupations["Names"].apply(remove_line_break)

    occ_empl_count = 0

    for occ_row in range(len(occupations)):
        if round(occupations["Change"].iloc[occ_row],1) > 0:
            occ_empl_count = occ_empl_count + 1

    para8 = "On a year-over-year basis, "

    if occ_empl_count == 0:
        para8 = para8+ "none of the occupations saw an increase in employment (Chart 5)."
    elif occ_empl_count == 1:
        para8 = para8+ "only " + occupations['Paragraph Names'].iloc[0]+ ' saw an increase in employment (Chart 5).'
    elif occ_empl_count == 2:
        para8 = para8+ occupations['Paragraph Names'].iloc[0]+ ", and "+occupations['Paragraph Names'].iloc[1]+ ' saw an increase in employment (Chart 5).'
    elif occ_empl_count == 3:
         para8 = para8+ "employment increased for: "+occupations['Paragraph Names'].iloc[0]+ "; "+occupations['Paragraph Names'].iloc[1] + "; and "+occupations['Paragraph Names'].iloc[2]+' (Chart 5).'
    else:
         para8 = para8+ "employment increased the most in: "+occupations['Paragraph Names'].iloc[0]+ "; "+occupations['Paragraph Names'].iloc[1]+ "; and "+occupations['Paragraph Names'].iloc[2]+' (Chart 5).'

    document.paragraphs[21].text = para8

except:
    para_errors.append("paragraph 8")


# In[ ]:





# In[ ]:





# In[30]:


# Chart 5
chart5_pos = 0.02
try:

    # Initialize the matplotlib figure
    # Size of chart
    #f, ax = plt.subplots(figsize=(1.4, 3.9))
    f, ax = plt.subplots(figsize=(1.4, 3.4))
    # Title

    ax.set_title('Chart 5 Year-over-Year Employment Growth\nBy Occupation - Seasonally Unadjusted',
                 size = 9, loc = 'left', x = -2, color = 'black')

    #ax.set_title('Chart 5 Year-over-Year Employment Growth\nBy Occupation - Seasonally Unadjusted', size = 9, x = 0)
    # Make Chart
    sns.barplot(x="Change", y="Names", data=occupations,palette = sns.color_palette([(0,170/255,210/255)],1),
                edgecolor = (0,0,0))
    # No axis titles
    ax.set( ylabel="",
           xlabel="")
    # Change text size of labels
    ax.tick_params(axis='both', which='major', labelsize=7, color = 'black')
    # Remove chart borders
    sns.despine(left=True, bottom=True)

    #Set x width
    
    if occupations['Change'].max() >0:
        x_max = occupations['Change'].max()+chart_text(occupations['Change'].max())/chart5_pos+ 0.125/chart5_pos
    else:
        x_max = 0
    if occupations['Change'].min() <0:
        x_min = occupations['Change'].min()-chart_text(occupations['Change'].min())/chart5_pos-0.25/chart5_pos
    else:
        x_min = 0
    ax.set_xlim(left = -abs(x_min)**1.05, right = x_max**1.05)
    
    chart5_pos2 = 5/(x_max-x_min)
    
    ###
    # Add labels
    for p in ax.patches:
        if p.get_width()<0:
            ax.text(p.get_width()-(chart_text(p.get_width())/chart5_pos2)- 0.25/chart5_pos2,
                    p.get_y()+ p.get_height()*0.7, num_to_string(p.get_width()), 
                fontsize=6, ha='center', va='bottom')
        else:
            ax.text(p.get_width()+(chart_text(p.get_width())/chart5_pos2)+ 0.125/chart5_pos2,
                    p.get_y()+ p.get_height()*0.7, num_to_string(p.get_width()), 
                fontsize=6, ha='center', va='bottom')

    plt.setp(ax.patches, linewidth=0.5)        
    ax.axvline(0, color = (209/255, 212/255,211/255), linewidth=1)        



    # Remove ticks and labels
    ax.set(xticklabels=[])
    ax.tick_params(bottom=False)

    ####



    # Save at a high resolution
    plt.savefig("AB Occupations.png",bbox_inches='tight', dpi = 1200)
    plt.close()


    p = document.paragraphs[22]
    r = p.add_run()
    r.add_picture("AB Occupations.png")

except:
    para_errors.append("chart 5")


# In[31]:


# Paragraph 9

try:

    # Sort regions from lowest to highest unemployment rate
    regions = df_p9.transpose()

    regions.columns = ["Current"]
    regions["Names"] = regions.index
    regions = regions.sort_values("Current")

    para9_part1 = "By economic regions, the seasonally unadjusted three-month moving average unemployment rate was lowest in "
    # Grab lowest value and region name
    para9_part2 = regions["Names"].iloc[0] + " ("+str(regions["Current"].iloc[0])+ " per cent) and highest in "
    # Grab highest value and region name
    para9_part3 = regions["Names"].iloc[-1] + " ("+str(regions["Current"].iloc[-1])+ " per cent) (Figure 1)."

    document.paragraphs[24].text = para9_part1 + para9_part2 + para9_part3

except:
    para_errors.append("paragraph 9")


# In[32]:


# Map (Figure 1)

try:

    # Setting fonts, text size and colours
    number_font = ImageFont.truetype("arial.ttf", 115)
    er_font = ImageFont.truetype("arial.ttf", 160)
    stroke_colour = (0,0,0)
    fill_colour = (255,255,255)
    outside_fill_colour = (0,0,0)

    # Load Alberta map image
    with Image.open('AB Map.png') as im:
        # create the canvas
        draw = ImageDraw.Draw(im)

        # Add title
        draw.multiline_text((0, 0),
                            "Figure 1 Alberta Labour Force Statistics by Economic Region\nSeasonally Unadjusted - Three-Month Moving Average",
                            fill = stroke_colour, font =ImageFont.truetype("arial.ttf", 130), align='left')

    #    draw.multiline_text((470, 0),
    #                        "Figure 1 Alberta Labour Force Statistics\nby Economic Region - Three-Month Moving Average",
    #                       fill = stroke_colour, font =ImageFont.truetype("arial.ttf", 130), align='center')


        # Locations


        #Banff, Jasper, RMH, ACGP, Peace River

        draw.multiline_text((600, 2400),
                            "Banff-Jasper-\nRocky Mountain House\nand Athabasca-Grande\nPrairie-Peace River",
                            fill = fill_colour,
                            font = er_font,
                            align='center')

        if df_map["BJ UR"].iloc[0]< 10:
            bj_space = "   "
        else:
            bj_space = " "

        draw.multiline_text((1152, 3000),
                            "PR: "+str(df_map["BJ PR"].iloc[0])+"%\nER: "+str(df_map["BJ ER"].iloc[0])+"%\nUR:"+ bj_space+str(df_map["BJ UR"].iloc[0])+"%",
                            fill = fill_colour,
                            font = number_font,
                            align='center')

        # Wood Buffalo Cold Lake
        draw.multiline_text((2600, 1300),
                            "Wood Buffalo-\nCold Lake",
                            fill = fill_colour,
                            font = er_font,
                            align='center')

        if df_map["WC UR"].iloc[0]< 10:
            wc_space = "   "
        else:
            wc_space = " "

        draw.multiline_text((2832, 1600),
                            "PR: "+str(df_map["WC PR"].iloc[0])+"%\nER: "+str(df_map["WC ER"].iloc[0])+"%\nUR:"+wc_space+str(df_map["WC UR"].iloc[0])+"%",
                            fill = fill_colour,
                            font = number_font,
                            align='center')


        # Camrose- Drumheller
        draw.multiline_text((2750, 4600),
                            "Camrose-\nDrumheller",
                            fill = fill_colour,
                            font = er_font,
                            align='center')

        if df_map["CD UR"].iloc[0]< 10:
            cd_space = "   "
        else:
            cd_space = " "

        draw.multiline_text((2868, 4900),
                            "PR: "+str(df_map["CD PR"].iloc[0])+"%\nER: "+str(df_map["CD ER"].iloc[0])+"%\nUR:"+cd_space+str(df_map["CD UR"].iloc[0])+"%",
                            fill = fill_colour,
                            font = number_font,
                            align='center')


        # +300 for height, + 190 for width
        # Lethbridge-Medicine Hat
        draw.multiline_text((2720, 6240),
                            "Lethbridge-\nMedicine Hat",
                            fill = fill_colour,
                            font = er_font,
                            align='center')

        if df_map["LM UR"].iloc[0]< 10:
            lm_space = "   "
        else:
            lm_space = " "


        draw.multiline_text((2912, 6540),
                            "PR: "+str(df_map["LM PR"].iloc[0])+"%\nER: "+str(df_map["LM ER"].iloc[0])+"%\nUR:"+lm_space+str(df_map["LM UR"].iloc[0])+"%",
                            fill = fill_colour,
                            font = number_font,
                            align='center')

        # Edmonton
        draw.multiline_text((50, 4800), "Edmonton",
                            fill = outside_fill_colour,
                            font = er_font,
                            align='center')

        if df_map["ED UR"].iloc[0]< 10:
            ed_space = "   "
        else:
            ed_space = " "

        draw.multiline_text((140, 4953), 
                            "PR: "+str(df_map["ED PR"].iloc[0])+"%\nER: "+str(df_map["ED ER"].iloc[0])+"%\nUR:"+ed_space+str(df_map["ED UR"].iloc[0])+"%",
                            fill = outside_fill_colour,
                            font = number_font,
                            align='center')

        # Line pointing to area
        draw.line([(775,5000),(2400,4500)], fill = stroke_colour, width =10)





        # Calgary
        draw.multiline_text((800, 6200),
                            "Calgary",
                            fill = outside_fill_colour, 
                            font = er_font,
                            align='center')

        if df_map["CA UR"].iloc[0]< 10:
            ca_space = "   "
        else:
            ca_space = " "

        draw.multiline_text((801, 6370),
                            "PR: "+str(df_map["CA PR"].iloc[0])+"%\nER: "+str(df_map["CA ER"].iloc[0])+"%\nUR:"+ca_space+str(df_map["CA UR"].iloc[0])+"%",
                            fill = outside_fill_colour,
                            font = number_font,
                            align='center')

        # Line pointing to area
        draw.line([(1400,6400),(2300,5700)], fill = stroke_colour, width =10)





        # Red Deer
        draw.multiline_text((450, 5500),
                            "Red Deer",
                            fill = outside_fill_colour,
                            font = er_font,
                            align='center')

        if df_map["RD UR"].iloc[0]< 10:
            rd_space = "   "
        else:
            rd_space = " "
        draw.multiline_text((519, 5653),
                            "PR: "+str(df_map["RD PR"].iloc[0])+"%\nER: "+str(df_map["RD ER"].iloc[0])+"%\nUR:"+rd_space+str(df_map["RD UR"].iloc[0])+"%",
                            fill = outside_fill_colour,
                            font = number_font,
                            align='center')

        # Line pointing to area
        draw.line([(1150,5700),(2400,5100)], fill = stroke_colour, width =10)



        # Labels At Bottom
        # 3848 width, 129 height
        draw.multiline_text((70, 7000),
                        "PR: Participaton Rate     ER: Employment Rate     UR: Unemployment Rate",
                        fill = stroke_colour, font =number_font)

        # Save map image
        #im.save("Map.png")
        im.save("Map with numbers.png")

    # Add image to document    
    p = document.paragraphs[25]
    r = p.add_run()
    r.add_picture("Map with numbers.png", width = Inches(3.75), height = Inches(6.95))

except:
    para_errors.append("figure 1")


# In[33]:


# Footer and meta data
try:
    
    footer_date = "©"+ str(datetime.date.today().year)+" Government of Alberta  |  "+datetime.date.today().strftime("%B %#d, %Y")+"  |  Jobs, Economy and Trade"

    document.sections[0].footer.paragraphs[2].text =  footer_date

    document.core_properties.title = "Labour Force Statistics - Alberta Highlights - "+ cur_m_text

    document.core_properties.author = 'Government of Alberta - Jobs, Economy and Trade'
    
    document.core_properties.subject = "Alberta Labour Force Statistics, Labour Force Participation Rate, Employment Rate, Unemployment Rate, Unemployment, Employment, Industry, Occupation, Region, Edmonton, Calgary, "+ cur_m_text
    
except:
    para_errors.append("footer and meta data")


# In[34]:


name='jet-lfs-highlights-'+file_date_text(m1)+'.docx'
document.save(name)
print("Program has been run. " +name+" is now available.")


# In[35]:


number_of_errors = len(para_errors)
if number_of_errors > 0:
    print_errors = 'The following paragraphs/charts have errors:'
    if number_of_errors != 1:
        error_count = 0
        for para in para_errors:
            error_count = error_count + 1
            if error_count == number_of_errors:
                print_errors = print_errors +" and "+ para +"."
            else:
                print_errors = print_errors +" " +para +","
    else:
        print_errors = '\nThe following paragraph/chart has errors: ' + para_errors[0]+"."
    print_errors = print_errors + "\nPlease correct the listed paragraphs/charts manually before sending to comms.\n\nAddionally, please double check the numbers and language in the other paragraphs."
else:
    print_errors = "\nThere were no errors in the paragraphs. Please double check the numbers, the language, and send the document to comms."

print(print_errors)


Program has been run. jet-lfs-highlights-2024-07.docx is now available.

There were no errors in the paragraphs. Please double check the numbers, the language, and send the document to comms.
